In [1]:
#| include: false

import duckdb
import pandas as pd
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

%sql CREATE OR REPLACE TABLE user_actions AS SELECT * FROM read_csv('00_data/sql/user_actions.csv', header=True, columns={'user_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE courier_actions AS SELECT * FROM read_csv('00_data/sql/courier_actions.csv', header=True, columns={'courier_id': 'INT', 'order_id': 'INT', 'action': 'VARCHAR', 'time': 'TIMESTAMP'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE orders AS SELECT * FROM read_csv('00_data/sql/orders.csv', header=True, columns={'order_id': 'INT', 'creation_time': 'TIMESTAMP', 'product_ids': 'INT[]'}, timestampformat='%d/%m/%y %H:%M');
%sql CREATE OR REPLACE TABLE users AS SELECT * FROM read_csv('00_data/sql/users.csv', header=True, columns={'user_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE couriers AS SELECT * FROM read_csv('00_data/sql/couriers.csv', header=True, columns={'courier_id': 'INT', 'birth_date': 'DATE', 'sex': 'VARCHAR'}, dateformat='%d/%m/%y');
%sql CREATE OR REPLACE TABLE products AS SELECT * FROM read_csv('00_data/sql/products.csv', header=True, columns={'product_id': 'INT', 'name': 'VARCHAR', 'price': 'DOUBLE'});

,Count
0,87


# Групування даних

## Оператор `GROUP BY`
Для групування даних в SQL використовується оператор `GROUP BY`. Він дозволяє групувати дані по одному або декільком стовпцям таблиці. При цьому вибірка даних буде містити лише унікальні значення з вказаних стовпців:

1. Спочатку в таблиці визначаються рядки, в яких у вказаному в `GROUP BY` стовпці є однакові значення.
2. Далі за цими значеннями записи об'єднуються у групи, причому у групі може бути навіть один запис.
3. Після цього над елементами цих груп, як правило, проводяться якісь операції за допомогою агрегатних функцій: наприклад, за допомогою `SUM()` обчислюється сума значень в якому-небудь стовпці в кожній групі:

```sql
SELECT column_1, SUM(column_2)
FROM table
GROUP BY column_1
```

4. Потім у результуючий запит потрапляє по одному унікальному запису зі стовпця, по якому здійснювалося групування. При цьому навпроти кожного такого значення відображається результат агрегації по відповідній групі, якщо агрегація проводилася.

Тут важливо зробити кілька уточнень:

- По-перше, групування виконується після фільтрації, тобто спочатку виконуються інструкції `WHERE` і лише потім дані групуються через `GROUP BY`.
- По-друге, до груп, що утворилися в результаті застосування `GROUP BY`, можна застосовувати відразу кілька агрегатних функцій (у тому числі до різних колонок).
- По-третє, групування можна робити відразу за новими полями, порахованими в `SELECT`: при цьому допускається використання в `GROUP BY` аліасу колонки, зазначеного в SELECT. Наступні два запити дадуть однаковий результат:

```sql
SELECT DATE(column_1) AS date, SUM(column_2)
FROM table
GROUP BY DATE(column_1)


SELECT DATE(column_1) AS date, SUM(column_2)
FROM table
GROUP BY date
```

- По-четверте, робити агрегацію після групування необов'язково. Якщо не вказувати агрегатну функцію, то запит поверне унікальні значення в стовпці, тобто той самий результат, як і оператор `DISTINCT`. Можете самостійно запустити наступний запит та переконатися:

```sql
SELECT user_id
FROM user_actions
GROUP BY user_id

SELECT DISTINCT user_id
FROM user_actions
```

До речі, питання про те, як можна відібрати унікальні значення, досвідчені аналітики люблять ставити на співбесідах своїм менш досвідченим колегам.

І нарешті, останнє важливе уточнення: під час використання групування колонки, вказаних у `SELECT`, повинні бути і `GROUP BY`, якщо вони використовуються у агрегатних функціях. Це обов'язкова умова, і якщо вона не буде виконана, база даних поверне помилку.

Наступний запит не працюватиме, оскільки в `GROUP BY` вказані не всі неагреговані колонки з блоку `SELECT`:

```sql
SELECT column_1, column_2, SUM(column_3)
FROM table
GROUP BY column_1
```

Водночас такий запит спрацює:

```sql
SELECT SUM(column_2)
FROM table
GROUP BY column_1
```

Зверніть увагу, що у цьому запиті у блоці `SELECT` немає колонки, вказаної у `GROUP BY`, тобто у зворотний бік правило не працює: якщо ми щось вказали в `GROUP BY`, це не обов'язково вказувати в `SELECT`. Інакше кажучи, виводити найменування груп необов'язково.

І ще: замість назв колонок у блоці `GROUP BY` можна використовувати номер колонки, вказаної у `SELECT`. Наприклад, наступні два запити еквівалентні:

```sql
SELECT column_1, column_2, SUM(column_3)
FROM table
GROUP BY column_1, column_2


SELECT column_1, column_2, SUM(column_3)
FROM table
GROUP BY 1, 2
```

При цьому номери колонок із `SELECT` також можна використовувати при сортуванні в операторі `ORDER BY`. Можете самі поекспериментувати із цим у наступних завданнях.

З теорією начебто розібралися, тепер приступимо до практики. Минулого разу ми вже рахували кількість кур'єрів жіночої статі та використовували для цього фільтрацію. Групування дозволить нам провести розрахунки відразу для двох статей.

:::: {.callout-note icon=false}
## Завдання
::: {#exr-sql-groupby-01}
<br>
За допомогою групування порахуйте кількість кур'єрів чоловічої та жіночої статі у таблиці `couriers`. Нову колонку з числом кур'єром назвіть `couriers_count`. Результат відсортуйте по цій колонці за зростанням.

Поля у результуючій таблиці: `sex`, `couriers_count`
:::
::::

In [2]:
#| code-fold: true
#| code-summary: "Рішення"

%%sql
SELECT sex,
       count(courier_id) as couriers_count
FROM   couriers
GROUP BY sex
ORDER BY couriers_count

,sex,couriers_count
0,female,1149
1,male,1674
